In [4]:
import lxml.etree as ET

In [6]:
file = 'vocabs/data/japbibthesaurus_03.rdf'

In [7]:
tree = ET.parse(file)

In [8]:
ns_skos = "http://www.w3.org/2004/02/skos/core#"
ns_rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"

In [9]:
descriptions = tree.findall('rdf:Description', namespaces={"rdf":ns_rdf})

In [11]:
concepts = []
for x in descriptions:
    concept ={}
    concept["id"] = x.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about']
    concept["notation"] = x.find('skos:notation', namespaces={"skos":ns_skos})
    concept["pref_labels"] = x.findall('skos:prefLabel', namespaces={"skos":ns_skos})
    concept["broader"] = x.findall('skos:broader', namespaces={"skos":ns_skos})
    concept["narrower"] = x.findall('skos:narrower', namespaces={"skos":ns_skos})
    concept["closeMatch"] = x.findall('skos:closeMatch', namespaces={"skos":ns_skos})
    concept["schemes"] =  x.findall('skos:inScheme', namespaces={"skos":ns_skos})
    concepts.append(concept)
print("# of Concetps: {}".format(len(concepts)))

# of Concetps: 1422


In [13]:
# create all conceptes mentioned in the skos-file
# <skos:broader rdf:resource="http://acdh.oeaw.ac.at/vocabs/professions/thepro/militar_2rd"/>
print(len(SkosConcept.objects.all()))
for x in concepts:
    temp_uri = x['id']
    temp_notation = temp_uri.split('/')[-1]
    temp_concept,_ = SkosConcept.objects.get_or_create(legacy_id=temp_uri, notation=temp_notation)
    try:
        temp_concept.pref_label=(x['pref_labels'][0].text)
        temp_concept.pref_label_lang=(x['pref_labels'][0].attrib['{http://www.w3.org/XML/1998/namespace}lang'])
    except:
        pass
    temp_concept.save()
    for y in x['pref_labels'][1:]:
        temp_label,_ = SkosLabel.objects.get_or_create(
            label=y.text,
            isoCode=y.attrib['{http://www.w3.org/XML/1998/namespace}lang'],
            label_type = "prefLabel"
        )
        temp_concept.label = [temp_label]
        temp_concept.save()
    for z in x['schemes']:
        temp_scheme,_ = SkosConceptScheme.objects.get_or_create(
            legacy_id=z.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource']
        )
        temp_scheme.dc_title = file
        temp_scheme.save()
        temp_concept.scheme=[temp_scheme]
    for y in x['broader']:
        broader_uri = y.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource']
        broader_notation =broader_uri.split('/')[-1]
        temp_broader,_ = SkosConcept.objects.get_or_create(legacy_id=broader_uri, notation=broader_notation)
        temp_broader.save()
        temp_concept.skos_broader=[temp_broader]
    for y in x['narrower']:
        narrower_uri = y.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource']
        narrower_notation =narrower_uri.split('/')[-1]
        temp_narrower,_ = SkosConcept.objects.get_or_create(legacy_id=narrower_uri, notation=narrower_notation)
        temp_narrower.save()
        temp_concept.skos_narrower=[temp_narrower]
    for y in x['closeMatch']:
        closeMatch_uri = y.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource']
        closeMatch_notation =closeMatch_uri.split('/')[-1]
        temp_closeMatch,_ = SkosConcept.objects.get_or_create(legacy_id=closeMatch_uri, notation=closeMatch_notation)
        temp_closeMatch.save()
        temp_concept.skos_closematch=[temp_closeMatch]
print(len(SkosConcept.objects.all()))

0
1010
